### Librerias

In [3]:
#instalacion
%pip install langchain_openai gradio chromadb pypdf ipywidgets langchain_community

  Using cached langchain_openai-0.3.3-py3-none-any.whl.metadata (2.7 kB)
  Using cached gradio-5.14.0-py3-none-any.whl.metadata (16 kB)
  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached langchain_community-0.3.16-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_core-0.3.33-py3-none-any.whl.metadata (6.3 kB)
  Using cached fastapi-0.115.8-py3-none-any.whl.metadata (27 kB)
  Using cached gradio_client-1.7.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.29.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.50b0-py3-none-any.whl.metadata (2.1 kB)
  Using cached opentelemetry_sdk-1.29.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached langchain-0.3.17-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_text_splitters-0.3.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_instrumentation_asgi-0.50b0-py3-none-any.whl.metadata (1.9 kB)
  Using c

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\inesc\\Bootcamp\\Module_3\\Proyecto_Final_Smartwaste_Madrid\\venv2\\Lib\\site-packages\\langchain_core-0.3.33.dist-info\\RECORD9h2qq6zr.tmp' -> 'c:\\Users\\inesc\\Bootcamp\\Module_3\\Proyecto_Final_Smartwaste_Madrid\\venv2\\Lib\\site-packages\\langchain_core-0.3.33.dist-info\\RECORD'
Check the permissions.


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
%pip install gradio


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached gradio-5.14.0-py3-none-any.whl.metadata (16 kB)
Using cached gradio-5.14.0-py3-none-any.whl (57.7 MB)


In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate 
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import gradio as gr

### Fuentes RAG

In [24]:
url = 'https://www.madrid.es/UnidadesDescentralizadas/Educacion_Ambiental/ContenidosBasicos/Publicaciones/Residuos_Limpieza/GuiaReducirResiduos/GuíaBuenasPrácticasReducirResiduos.pdf'
url2 = 'DATA/CS-RecogidaResiduos_Folleto2024.pdf'
url3 = 'https://www.madrid.es/UnidadesDescentralizadas/UDCEstadistica/Nuevaweb/Publicaciones/anuesta/nuevos/Anuario%20Estadístico%20Municipal/Anuario%20estadístico%202023/Capítulos/CAPITULO%206.pdf'
url4 = 'https://www.icv.csic.es/prevencion/Documentos/residuos/puntos.pdf'
url5 =  'https://www.redcimas.org/wordpress/wp-content/uploads/2013/02/gestionresiduos_comparativa.pdf'
url6 = 'https://conerrede.es/wp-content/uploads/2022/08/dossier-informativo_ayto-madrid.pdf'
url7 = 'https://www.ucm.es/data/cont/docs/3-2019-04-08-PUNTOS%20LIMPIOS%20AYTO.%20MADRID.pdf'
url8 = 'https://transparencia.madrid.es/FWProjects/transparencia/PlanesYMemorias/Planes/LimpiezaResiduos/ficheros/ProyectoEstrategiaResiduos2030.pdf'

### Configuracion

In [25]:
API_KEY = ''
llm = ChatOpenAI(model='gpt-3.5-turbo-0125' , temperature= 0 , api_key=API_KEY)
emmbeding = OpenAIEmbeddings(api_key= API_KEY)

### Corpus

In [26]:
urls = [url, url3, url4, url5, url6, url7, url8] #un libro de libros

In [27]:
pages = []
for url in urls:
    pdf = PyPDFLoader(url)
    pages += pdf.load_and_split()

In [28]:
len(pages)

524

In [29]:
pages[410]

Document(metadata={'source': 'https://transparencia.madrid.es/FWProjects/transparencia/PlanesYMemorias/Planes/LimpiezaResiduos/ficheros/ProyectoEstrategiaResiduos2030.pdf', 'page': 167, 'page_label': '168'}, page_content='Proyecto de Estrategia de prevención y gestión de residuos domésticos y comerciales  \nde la ciudad de Madrid – 2030   \nPg. 155  de 266  \n \n  \n  \nPROGRAMA 2: Medidas para la adaptación del sistema de contenerización \ny del sistema de recogida y transporte de residuos a las fracciones \nseparadas en origen  \n           \nAcción  13 Fomento de los puntos limpios    \nMEDIDAS DE ACOMPAÑAMIENTO  \nCampañas de comunicación y sensibilización como ReMad.  \n  \nINDICADORES DE SEGUIMIENTO  \n\uf0b7 Cantidad total de residuos recogidos respecto al a ño anterior en puntos limpios por tipología de \nresiduo en toneladas (t). \n\uf0b7 Cantidad total de residuos destinados a la reutili zación provenientes de puntos limpios con relación al \naño anterior en toneladas (t). \n

# Vectorstore 

In [20]:
%pip install chromadb

  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.29.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.50b0-py3-none-any.whl.metadata (2.1 kB)
  Using cached opentelemetry_instrumentation_asgi-0.50b0-py3-none-any.whl.metadata (1.9 kB)
Using cached chromadb-0.6.3-py3-none-any.whl (611 kB)
Using cached opentelemetry_exporter_otlp_proto_grpc-1.29.0-py3-none-any.whl (18 kB)
Using cached opentelemetry_instrumentation_fastapi-0.50b0-py3-none-any.whl (12 kB)
Using cached opentelemetry_instrumentation_asgi-0.50b0-py3-none-any.whl (16 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
# Initialize the vectorstore with persistence
vectorstore = Chroma.from_documents(
    documents=pages,
    embedding=emmbeding,
    persist_directory="C:/Users/inesc/Bootcamp/Module_3/Proyecto_Final_Smartwaste_Madrid/Smartwaste_Streamlit_V2/vector_store"  # Specify directory
)

# Persist the vectorstore to disk
vectorstore.persist()

# Create a retriever
retriever = vectorstore.as_retriever()

# Prompt Template

In [31]:
 template = """
          ###INSTRUCCIONES:
          Eres un asistente virtual en tareas de concienciacion del reciclaje y las buenas practicas medioambientales y tu objetivo es recopilar y dar respuesta segun la informacion que tienes a las preguntas que te planteen los usuarios.Seimpre con un tono amigable y pedagogico.

          En tu respuesta, SIEMPRE:
          (0)Se un lector atento a los detalles: Lee la pregunta y el contexto, entiende ambos antes de responder
          (1)Tu respuesta comienza con un tono amigable, asegúrate de que entiendes la pregunta del usuario correctamente, en caso negativo, pregunta al usuario para obtener más información.
          (2)Si el contexto te permite responder a la pregunta, escribe una respuesta detallada, útil y fácil de entender, CON FUENTES REFERENCIADAS EN EL TEXTO. SI NO: no puedes encontrar la respuesta, responde con una explicación, comenzando con: "No pude encontrar la información en los documentos a los que tengo acceso".
          (3)Debajo de tu respuesta, proporciona un enlace a la fuente referenciada( Es decir, párrafos legales, artículos, etc.) y una breve explicación de por qué la fuente es relevante.
          (4)Ahora que ya tienes tu respuesta, revísala para asegurarte de que sea clara, precisa y educada y cumple con los puntos anteriores.

          ###PIENSA PASO POR PASO

            Responde a la siguiente pregunta utilizando el contexto proporcionado.
              ### Question: {question} ###
              ### Context: {context} ###
              ### Respuesta 'util con Fuentes:        
"""

prompt = PromptTemplate.from_template(template)

## Chain

In [32]:
chain = (
    {'context':retriever,
     'question':RunnablePassthrough()}
     | prompt
     | llm
     | StrOutputParser()
)

In [33]:
def get_asnwer(question):
    return chain.invoke(question)

In [34]:
get_asnwer('donde puedo encotrar un punto limpio en carabanchel')

'¡Hola! Para encontrar un punto limpio en Carabanchel, puedes dirigirte a la siguiente dirección:\n\nCalle Cidro, 5 (Barrio Buenavista) 28044 Madrid\n\nAdemás, te dejo los horarios de los diferentes puntos limpios en Carabanchel:\n- C/ Carlos Daban, frente 28. Horario: lunes de 14 a 15:30 horas.\n- Avda. de Oporto, 96. Horario: lunes de 15:45 a 17:15 horas.\n- Avda. Plaza de Toros, frente 8. Horario: lunes de 17:30 a 19 horas.\n- C/ Mazarambroz, 8. Horario: lunes de 19:15 a 20:45 horas.\n\nRecuerda que los puntos limpios son lugares donde puedes llevar residuos especiales para su correcta gestión y reciclaje. ¡Espero que esta información te sea útil!\n\nPara más detalles, puedes consultar el documento oficial del Ayuntamiento de Madrid donde se detallan los puntos limpios en la ciudad: [Enlace al documento](https://www.ucm.es/data/cont/docs/3-2019-04-08-PUNTOS%20LIMPIOS%20AYTO.%20MADRID.pdf). Esta fuente es relevante ya que proporciona información oficial y detallada sobre la ubicación

In [16]:
iface = gr.Interface(fn=get_asnwer, inputs=gr.Textbox(
    value="Pregúntame lo que necesites"),
    live=False, 
    outputs="markdown",  
    title="Inteligencia sobre documentación mediante LLM",
    description="Puedes preguntar cualquier pregunta sobre migración",
    theme=gr.themes.Soft(),
    allow_flagging="never",)

iface.launch()

c:\Users\inesc\Bootcamp\Module_3\Proyecto_Final_Smartwaste_Madrid\Smartwaste_Streamlit_V2\venv2\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
